# Openstreetmap et graphes

Utilisation d'openstreetmap pour construire des graphes de réseaux routiers.

Ceci permettra de trouver des chemins de longueurs minimum.

In [ ]:
import osmnx as ox

On utilise la bibliothèque OSMnx qui connecte OSM et networkx. Jetez un coup d'oeil aux divers exemples fournis, ils sont éclairant sur les capacités de cette bibliothèque :

<https://github.com/gboeing/osmnx-examples/tree/main/notebooks>

In [ ]:
Marseille = ox.graph.graph_from_address('Marseille, France', 1000)

la bibliothèque [osmnx](https://osmnx.readthedocs.io/) récupère d'openstreetmap le graphe des routes de Marseille (on le verra, c'est juste le centre).

Le type de graphe utilisé est un [`MultiDiGraph`](https://networkx.org/documentation/stable/reference/classes/multidigraph.html), c'est l'équivalent d'un multi-graphe (il peut y avoir plusieurs arêtes entre deux sommets) mixte (les arêtes peuvent être dirigées (route à sens unique) ou non (routes à double sens)).

In [ ]:
type(Marseille)

## Dessin

In [ ]:
import matplotlib.pyplot as plt

### Directement

In [ ]:
ox.plot_graph(Marseille)

Vous devriez voir apparaître (après un certain temps) une fenêtre avec un graphe où l'on devine le [vieux-port de Marseille](https://www.google.fr/maps/@43.2944646,5.3601266,16z).

### Avec matplotlib

On peut aussi utiliser la figure de matplotlib que l'on peut paramétrer ensite.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12)) 

ax.set_facecolor("#111111")

ox.plot_graph(Marseille, ax=ax, show=False)
plt.show()

### Réseau routier

In [ ]:
ox.plot.plot_figure_ground(Marseille)

## Obtenir des graphes

Il existe plusieurs façons d'obtenir des graphes de réseaux routiers, nous allons en voir trois.

> Il est aussi possible d'obtenir d'autres types de réseaux. Référez vous au paramètre `custom_filter` dans la documentation.

### A partir d'une adresse

<https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.graph.graph_from_address>


In [ ]:
ecm = ox.graph.graph_from_address('Ecole centrale marseille', dist=2000)

fig, ax = ox.plot_graph(ecm, show=False)
plt.show()

Diminuons la distance pour *reconnaître* l'école.

In [ ]:
ecm = ox.graph.graph_from_address('Ecole centrale marseille', dist=500)

fig, ax = ox.plot_graph(ecm, show=False)
plt.show()

Notez que si l'on cherche centrale med, cela ne parche pas.

### A partir d'une boite `bbox`

<https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.graph.graph_from_bbox>

In [ ]:
marseille_en_grand = ox.graph.graph_from_bbox((43.388, 43.168, 5.498, 5.295), network_type='drive')

fig, ax = ox.plot_graph(marseille_en_grand, show=False)
plt.show()

On peut utiliser <http://norbertrenner.de/osm/bbox.html> pour construire nos `bbox`. 

Faisons celle de l'école, on obtient un truc du genre : 

```
5.43466,43.33865,5.44056,43.34519(left,bottom,right,top)
```

Attention aux coordonnées, la doc nous dis que l'ordre est nord, sud, est, ouest.

In [ ]:
ecm = ox.graph.graph_from_bbox((43.34519, 43.33865, 5.44056, 5.43466), network_type='drive')

fig, ax = ox.plot_graph(ecm, show=False)
plt.show()

On a que les route. Pour voir tous les chemins, on utilise tout le réseau de routes (c'est le paramètre par défaut) :

In [ ]:
ecm = ox.graph.graph_from_bbox((43.34519, 43.33865, 5.44056, 5.43466))

fig, ax = ox.plot_graph(ecm, show=False)
plt.show()

### A partir de coordonnées GPS

<https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.graph.graph_from_point>


In [ ]:
ailefroide = ox.graph.graph_from_point((44.8833273, 6.444307), dist=3000, network_type='all')

fig, ax = ox.plot_graph(ailefroide, show=False)
plt.show()

On peut facilement voir où c'est grace à google maps : <https://www.google.fr/maps/@44.8833273,6.444307,13z>

Les 3 paramètres sont :

1. latitude
2. longitude
3. zoom

### Type de réseau

le paramètre `network_type` permet de déterminer quel réseau routier est utilisé par défaut. D'après la documentation, il y a plusieurs possiblités :

- `"all_private"` (par défaut, tout y compris les chemins privés)
- `"all"`
- `"bike"`
- `"drive"`
- `"drive_service"`
- `"walk"`

Comparons l'ecm à vélo et en voiture :

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12), ncols=2) 

ax[0].set_facecolor("#111111")
ax[1].set_facecolor("#111111")
ax[0].set_title("Voiture")
ax[1].set_title("Vélo")

ox.plot_graph(ox.graph_from_point((43.3426309, 5.4350088), dist=750, network_type='drive'), ax=ax[0], show=False)
ox.plot_graph(ox.graph_from_point((43.3426309, 5.4350088), dist=750, network_type='bike'), ax=ax[1], show=False)

plt.show()

On remarque les routes sont considérées comme cyclables

## Obtenir des batiments

En plus du réseau routier, openstreetmap met à disposition des features :

<https://wiki.openstreetmap.org/wiki/Map_features>

Qui sont tout les centres d'intérets (building, arrêt de bus, etc)

In [ ]:
ecm_features = ox.features_from_point((43.3426309, 5.4350088), tags = {'building': True}, dist=750)

In [ ]:
type(ecm_features)

In [ ]:
ecm_features.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12)) 

ecm_features.plot(ax=ax)

plt.show()

On remarque qu'il manque l'amphi JouLe

## Données


In [ ]:
ailefroide.graph

Le CRS [epsg:4326](https://epsg.io/4326) est le crs du GPS, classique. 

### CRS

On peut facilement en changer en utilisant [pyproj](https://pyproj4.github.io/) :

In [ ]:
from pyproj import CRS

In [ ]:
ailefroide_mercator = ox.projection.project_graph(ailefroide, to_crs=CRS.from_string("epsg:3785"))

Sur un si petit graphe, la forme ne change pas vraiment :

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12), ncols=2) 

ax[0].set_facecolor("#111111")
ax[1].set_facecolor("#111111")
ax[0].set_title("Mercator")
ax[1].set_title("GPS")

ox.plot_graph(ailefroide_mercator, ax=ax[0], show=False)
ox.plot_graph(ailefroide, ax=ax[1], show=False)

plt.show()

En revanche, les coordonnées sont très différentes.

In [ ]:
ax[0].get_xlim(), ax[0].get_ylim()  # mercator

In [ ]:
ax[1].get_xlim(), ax[1].get_ylim()  # GPS

### Sommets et arêtes

In [ ]:
len(ailefroide.nodes)

In [ ]:
len(ailefroide.edges)

Le graphe a 189 sommets et 453 arêtes (le 7/1/24). 

Chaque sommet est un numéro (comme `268931860`) :

In [ ]:
[n for n in ailefroide.nodes][:10]  # 10 premiers sommets

In [ ]:
ailefroide.nodes[268931832]

les arêtes sont des triplets `(sommet origine, sommet arrivé, numéro d'arête)`. Le numéro d'arête est par défaut 0 (c'est le cas général s'il n'y a qu'une arête par couple de sommet) :

In [ ]:
[e for e in ailefroide.edges][:10]  # 10 premières arêtes

In [ ]:
ailefroide.edges[(268931832, 6643877819, 0)]

Pour connaître le sommet associé à une coordonnée, on utilise les fonctions :

- [`get_nearest_nodes`](https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.distance.nearest_nodes)
- [`get_nearest_edges`](https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.distance.nearest_edges)

Par exemple (**faite attention à l'ordre des coordonnées**) :

In [ ]:
sommet = ox.distance.nearest_nodes(ailefroide, 6.41556, 44.91679)
arete = ox.distance.nearest_edges(ailefroide, 6.41556, 44.91679)

In [ ]:
sommet

In [ ]:
arete

In [ ]:
ailefroide.nodes[sommet]

C'est à dire que le sommet le plus proche est aux coordonnées GPS (44.8707699, 6.4812867) et est de degré 3. C'est [pré de madame Carle](https://fr.wikipedia.org/wiki/Pr%C3%A9_de_Madame_Carle)

In [ ]:
fig, ax = ox.plot_graph(ailefroide, show=False,
                        node_color=["red" if n == sommet else "white" for n in ailefroide.nodes],
                        node_size=[100 if n == sommet else 15 for n in ailefroide.nodes]
                       )

plt.show()

In [ ]:
ailefroide.edges[arete]

C'est une route à sens unique de 5km de longueur (c'est une descente de ski depuis le sommet du pelvoux)

In [ ]:
fig, ax = ox.plot_graph(ailefroide, show=False,
                        edge_color=["red" if e == arete else "white" for e in ailefroide.edges],
                        edge_linewidth=[3 if e == arete else 1 for e in ailefroide.edges]
                       )

plt.show()

Il doit y avoir plusieurs routes qui passent par notre route et donc selon l'ordre d'affichage notre arete est en-dessous des autres. 

Utilisons une autre fonction : [`plot_graph_route`](https://osmnx.readthedocs.io/en/stable/user-reference.html#osmnx.plot.plot_graph_route) :

In [ ]:
fig, ax = ox.plot_graph_route(ailefroide, [arete[0], arete[1]])

On peut mettre le réeau routier en surbrillance :

In [ ]:
fig, ax = ox.plot_figure_ground(ailefroide, dist=3500, show=False)

plt.show()

Mais le système de coordonnée est tout autre :

In [ ]:
ailefroide.nodes[sommet]

In [ ]:
ailefroide_mercator.nodes[sommet]

## Fond de cartes

Pour ajouter un fond de carte à notre graphe il faut :

1. un fond de carte
2. s'assurer que la carte et le graphe ont le même crs
3. transformer le graphe en geodataframe
4. supperposer les deux dessins.

On utilise [contextily](https://contextily.readthedocs.io/) pour les fond de cartes (au format GPS, on a donc pas à transformer le CRS de notre graphe) :

In [ ]:
import contextily as ctx

Conversion du graphe en 2 Geodataframes, l'un pour les sommet, l'autre pour les arêtes :

In [ ]:
ailefroide_sommets_gdf, ailefroide_arêtes_gdf = ox.graph_to_gdfs(ailefroide)

In [ ]:
type(ailefroide_arêtes_gdf)

In [ ]:
ailefroide_arêtes_gdf.head()

Si on ne veut que les arêtes on peut aussi :

In [ ]:
ailefroide_gdf = ox.graph_to_gdfs(ailefroide, nodes=False)

Cette conversion au format geopandas des arêtes du graphe permet de travailler directement avec des geodataframes.

In [ ]:
ailefroide_gdf.index[0]

In [ ]:
ailefroide_gdf.iloc[0]

On peut alors avoir directement accès à toutes les manipulations graphiques que l'on a vu précédemment, en comme la supperposition de la carte et du graphe. 

In [ ]:
import xyzservices.providers as xyz

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10)) 


ailefroide_gdf.plot(ax=ax)
ctx.add_basemap(ax, crs=ailefroide_gdf.crs.to_string(), 
                source=xyz.GeoportailFrance.plan)

plt.plot()

Ce genre d'approche vous permer de combiner de créer des cartes très détaillées, puisque l'on peut ajouter ce que l'on veut, en particuliers les features associées à la carte :

In [ ]:
aiefroide_features = ox.features_from_point((44.8833273, 6.444307), dist=3000,
                                            tags = {'amenity': True})

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10)) 

ailefroide_gdf.plot(ax=ax)
ctx.add_basemap(ax, crs=ailefroide_gdf.crs.to_string(), 
                source=xyz.GeoportailFrance.plan)
aiefroide_features.plot(ax=ax, color="red")

plt.plot()

## Chemins

On veut aller de l'école à deep pour acheter du café.

> TBD : trouver dans marseille en grands le sommet le plus proche de l'ecm et de deep
> puis faire un chemin le plus court et l'afficher

### Détermination des sommets de départ et d'arrivée

Si l'on ne connait pas les coordonnées GPS des points de départ et d'arrivée d'arrivées on puet les trouver en utilisant OSM via ;la bibliothèque [`geopy`](https://geopy.readthedocs.io/) (qui devrait déjà être installée via une dépendance).

In [ ]:
import geopy

from geopy.geocoders import Nominatim  # décodeur OSM
geolocator = Nominatim(user_agent="juterlab cours ecm")

Faisons une requête pour trouver le départ :

In [ ]:
départ = geolocator.geocode("Ecole centrale marseille, france")

In [ ]:
départ

In [ ]:
type(départ)

En lisant la [doc](https://geopy.readthedocs.io/en/latest/index.html#geopy.location.Location) on trouve les coordonnées recherchées :

In [ ]:
départ.point

In [ ]:
arrivée = geolocator.geocode("café deep, marseille, france")

In [ ]:
arrivée

### Sommet assocées

Trouvons les sommets associés sur le graphe (attention à l'ordre des coordonnées) :

In [ ]:
départ_sommet = ox.nearest_nodes(marseille_en_grand, départ.point[1], départ.point[0]) 
arrivée_sommet = ox.nearest_nodes(marseille_en_grand, arrivée.point[1], arrivée.point[0]) 

In [ ]:
départ_sommet, arrivée_sommet

In [ ]:
fig, ax = ox.plot_graph(marseille_en_grand, show=False,
                        node_color=["red" if n in (départ_sommet, arrivée_sommet) else "white" for n in marseille_en_grand.nodes],
                        node_size=[100 if n in (départ_sommet, arrivée_sommet) else 15 for n in marseille_en_grand.nodes]
                       )

plt.show()

### Route

On utilise [`shortest_path`](https://osmnx.readthedocs.io/en/stable/user-reference.html#osmnx.routing.shortest_path) qui permet de trouver un chemin le plus court selon le poid considéré. Allons-y le plus vite possible.

In [ ]:
route = ox.shortest_path(marseille_en_grand, départ_sommet, arrivée_sommet, weight='travel_time')

La fonction rend une liste de sommets :

In [ ]:
len(route)

In [ ]:
route[:10]  # les dix premiers sommets

Que l'on peut afficher :

In [ ]:
fig, ax = ox.plot_graph_route(marseille_en_grand, route)

Le chemin le plus court prend la L2 jusqu'au vieux port, mais ne compte pas le temps pour trouver une place de parking gratuite.

## Coordonnées géographiques

On a déjà vu les formes géométriques Point et Polygon du format geojson. Les arêtes du graphe utilisent une autre primitive, les [`LineString`](https://shapely.readthedocs.io/en/stable/reference/shapely.LineString.html).

Considérons uniquement le geodataframe des arêtes utilisées pour aller de l'ecm à deep. Pour celà on va travailler par étapes :

1. créer le geodataframe des arêtes du graphe de marseille
2. trouver les arêtes du chemins (notre route est une suite de sommets
3. restreindre le geodataframe à ces arêtes

In [ ]:
marseille_en_grand_gdf = ox.graph_to_gdfs(marseille_en_grand, nodes=False)

In [ ]:
arêtes = [(route[i], route[i+1], 0) for i in range(len(route)-1)]

In [ ]:
chemin = marseille_en_grand_gdf.loc[marseille_en_grand_gdf.index.isin(arêtes)]

In [ ]:
chemin

Il y a 61 lignes à notre geodataframe (ce qui est cohérent avec les 62 sommets de la route) que l'on peut représenter graphiquement :

In [ ]:
fig, ax = plt.subplots(figsize=(20,12))

chemin.plot(ax=ax)

plt.show()

Comparons ce dessoin à ce que l'on aurait eu si l'on avait juste utilisé des segments entre les différents sommets 

In [ ]:
from shapely.geometry import Point, LineString

In [ ]:
rows = []
for i in range(len(route) - 1):
    x = Point(marseille_en_grand.nodes[route[i]]['x'], marseille_en_grand.nodes[route[i]]['y'])
    y = Point(marseille_en_grand.nodes[route[i + 1]]['x'], marseille_en_grand.nodes[route[i + 1]]['y'])
    row = {'orig': route[i], 
           'dest': route[i+1],
           'geometry': LineString([x, y])}
    rows.append(row)

In [ ]:
import geopandas as gpd

In [ ]:
gdf = gpd.GeoDataFrame(rows)

On a crée un geodataframe dont les LineString sont des segments :

In [ ]:
gdf

In [ ]:
fig, ax = plt.subplots(figsize=(20,12), ncols=2)

ax[0].set_title("Segments")
ax[1].set_title("Courbes")

gdf.plot(ax=ax[0])
chemin.plot(ax=ax[1])

plt.show()

Finissons cette partie en représentant les deux chemins sur la même carte :

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10)) 

ax.axis(False)

chemin.plot(ax=ax, color="red")
gdf.plot(ax=ax, color="blue")
ctx.add_basemap(ax, crs=chemin.crs.to_string())


plt.plot()